# <font color='blue'> Step 1 - Scraping </font>

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Nasa Mars News

In [13]:
#URL for NASA Mars News
url = 'https://redplanetscience.com/'
browser.visit(url)

In [14]:
# HTML object
html = browser.html
    
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Retrieve all elements that has article information
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

A Year of Surprising Science From NASA's InSight Mars Mission
A batch of new papers summarizes the lander's findings above and below the surface of the Red Planet.


## JPL Mars Space Images - Featured Image

In [15]:
#URL for JPL Mars Space Images - Featured Image
space_url = 'https://spaceimages-mars.com/'
browser.visit(space_url)

In [17]:
#Use Splinter to navigate the site to find Featured Mars Image
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Retrieve all elements that article information
div = soup.find('div', class_='header')
a = div.find('a', class_='showimg fancybox-thumbs')
href = a['href']
featured_image_url = str(space_url) + str(href)
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars3.jpg


In [18]:
browser.quit()

## Mars Facts

In [10]:
mars_facts_url = 'https://galaxyfacts-mars.com'
tables = pd.read_html(mars_facts_url)
mars_df = tables[0]
mars_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C
